In [3]:
import feature_extractor as fe 
from os import listdir
import scipy.io as spio
import numpy as np
import dataset

In [98]:
patient_number = 7
data, labels = dataset.get_data(patient_number)

307 number of points
1/307: extracting patient_7_197.mat
2/307: extracting patient_7_183.mat
3/307: extracting patient_7_154.mat
4/307: extracting patient_7_140.mat
5/307: extracting patient_7_168.mat
6/307: extracting patient_7_26.mat
7/307: extracting patient_7_32.mat
8/307: extracting patient_7_6.mat
9/307: extracting patient_7_222.mat
10/307: extracting patient_7_236.mat
11/307: extracting patient_7_237.mat
12/307: extracting patient_7_223.mat
13/307: extracting patient_7_7.mat
14/307: extracting patient_7_33.mat
15/307: extracting patient_7_27.mat
16/307: extracting patient_7_169.mat
17/307: extracting patient_7_141.mat
18/307: extracting patient_7_155.mat
19/307: extracting patient_7_182.mat
20/307: extracting patient_7_196.mat
21/307: extracting patient_7_180.mat
22/307: extracting patient_7_194.mat
23/307: extracting patient_7_143.mat
24/307: extracting patient_7_157.mat
25/307: extracting patient_7_19.mat
26/307: extracting patient_7_31.mat
27/307: extracting patient_7_25.mat


238/307: extracting patient_7_259.mat
239/307: extracting patient_7_265.mat
240/307: extracting patient_7_271.mat
241/307: extracting patient_7_270.mat
242/307: extracting patient_7_264.mat
243/307: extracting patient_7_258.mat
244/307: extracting patient_7_48.mat
245/307: extracting patient_7_74.mat
246/307: extracting patient_7_60.mat
247/307: extracting patient_7_106.mat
248/307: extracting patient_7_112.mat
249/307: extracting patient_7_304.mat
250/307: extracting patient_7_175.mat
251/307: extracting patient_7_161.mat
252/307: extracting patient_7_149.mat
253/307: extracting patient_7_13.mat
254/307: extracting patient_7_203.mat
255/307: extracting patient_7_217.mat
256/307: extracting patient_7_216.mat
257/307: extracting patient_7_202.mat
258/307: extracting patient_7_12.mat
259/307: extracting patient_7_148.mat
260/307: extracting patient_7_160.mat
261/307: extracting patient_7_174.mat
262/307: extracting patient_7_189.mat
263/307: extracting patient_7_162.mat
264/307: extracti

171/2400: extracting patient_7_355.mat
172/2400: extracting patient_7_1780.mat
173/2400: extracting patient_7_1958.mat
174/2400: extracting patient_7_2289.mat
175/2400: extracting patient_7_1794.mat
176/2400: extracting patient_7_369.mat
177/2400: extracting patient_7_2074.mat
178/2400: extracting patient_7_180.mat
179/2400: extracting patient_7_194.mat
180/2400: extracting patient_7_1569.mat
181/2400: extracting patient_7_2060.mat
182/2400: extracting patient_7_1541.mat
183/2400: extracting patient_7_802.mat
184/2400: extracting patient_7_2048.mat
185/2400: extracting patient_7_1227.mat
186/2400: extracting patient_7_1233.mat
187/2400: extracting patient_7_816.mat
188/2400: extracting patient_7_1555.mat
189/2400: extracting patient_7_625.mat
190/2400: extracting patient_7_143.mat
191/2400: extracting patient_7_157.mat
192/2400: extracting patient_7_631.mat
193/2400: extracting patient_7_619.mat
194/2400: extracting patient_7_1582.mat
195/2400: extracting patient_7_1596.mat
196/2400: e

381/2400: extracting patient_7_2307.mat
382/2400: extracting patient_7_1168.mat
383/2400: extracting patient_7_581.mat
384/2400: extracting patient_7_2313.mat
385/2400: extracting patient_7_1829.mat
386/2400: extracting patient_7_1197.mat
387/2400: extracting patient_7_1183.mat
388/2400: extracting patient_7_218.mat
389/2400: extracting patient_7_556.mat
390/2400: extracting patient_7_1815.mat
391/2400: extracting patient_7_230.mat
392/2400: extracting patient_7_224.mat
393/2400: extracting patient_7_1801.mat
394/2400: extracting patient_7_542.mat
395/2400: extracting patient_7_225.mat
396/2400: extracting patient_7_543.mat
397/2400: extracting patient_7_1800.mat
398/2400: extracting patient_7_1814.mat
399/2400: extracting patient_7_557.mat
400/2400: extracting patient_7_231.mat
401/2400: extracting patient_7_219.mat
402/2400: extracting patient_7_1182.mat
403/2400: extracting patient_7_1828.mat
404/2400: extracting patient_7_1196.mat
405/2400: extracting patient_7_2312.mat
406/2400: e

593/2400: extracting patient_7_636.mat
594/2400: extracting patient_7_1585.mat
595/2400: extracting patient_7_178.mat
596/2400: extracting patient_7_2098.mat
597/2400: extracting patient_7_1591.mat
598/2400: extracting patient_7_839.mat
599/2400: extracting patient_7_2073.mat
600/2400: extracting patient_7_187.mat
601/2400: extracting patient_7_1208.mat
602/2400: extracting patient_7_193.mat
603/2400: extracting patient_7_2067.mat
604/2400: extracting patient_7_805.mat
605/2400: extracting patient_7_1546.mat
606/2400: extracting patient_7_1220.mat
607/2400: extracting patient_7_1234.mat
608/2400: extracting patient_7_1552.mat
609/2400: extracting patient_7_811.mat
610/2400: extracting patient_7_346.mat
611/2400: extracting patient_7_1963.mat
612/2400: extracting patient_7_420.mat
613/2400: extracting patient_7_434.mat
614/2400: extracting patient_7_1977.mat
615/2400: extracting patient_7_352.mat
616/2400: extracting patient_7_1787.mat
617/2400: extracting patient_7_1793.mat
618/2400: e

...

1455/2400: extracting patient_7_1885.mat
1456/2400: extracting patient_7_1675.mat
1457/2400: extracting patient_7_1113.mat
1458/2400: extracting patient_7_288.mat
1459/2400: extracting patient_7_1107.mat
1460/2400: extracting patient_7_1661.mat
1461/2400: extracting patient_7_2368.mat
1462/2400: extracting patient_7_2383.mat
1463/2400: extracting patient_7_511.mat
1464/2400: extracting patient_7_1852.mat
1465/2400: extracting patient_7_277.mat
1466/2400: extracting patient_7_263.mat
1467/2400: extracting patient_7_1846.mat
1468/2400: extracting patient_7_2397.mat
1469/2400: extracting patient_7_505.mat
1470/2400: extracting patient_7_539.mat
1471/2400: extracting patient_7_538.mat
1472/2400: extracting patient_7_262.mat
1473/2400: extracting patient_7_504.mat
1474/2400: extracting patient_7_2396.mat
1475/2400: extracting patient_7_1847.mat
1476/2400: extracting patient_7_1853.mat
1477/2400: extracting patient_7_510.mat
1478/2400: extracting patient_7_2382.mat
1479/2400: extracting pati

1671/2400: extracting patient_7_475.mat
1672/2400: extracting patient_7_1936.mat
1673/2400: extracting patient_7_1088.mat
1674/2400: extracting patient_7_313.mat
1675/2400: extracting patient_7_307.mat
1676/2400: extracting patient_7_1922.mat
1677/2400: extracting patient_7_461.mat
1678/2400: extracting patient_7_449.mat
1679/2400: extracting patient_7_2224.mat
1680/2400: extracting patient_7_2230.mat
1681/2400: extracting patient_7_1739.mat
1682/2400: extracting patient_7_2218.mat
1683/2400: extracting patient_7_1711.mat
1684/2400: extracting patient_7_1077.mat
1685/2400: extracting patient_7_1063.mat
1686/2400: extracting patient_7_1705.mat
1687/2400: extracting patient_7_1075.mat
1688/2400: extracting patient_7_1713.mat
1689/2400: extracting patient_7_488.mat
1690/2400: extracting patient_7_1707.mat
1691/2400: extracting patient_7_1061.mat
1692/2400: extracting patient_7_1049.mat
1693/2400: extracting patient_7_2226.mat
1694/2400: extracting patient_7_2232.mat
1695/2400: extracting 

1888/2400: extracting patient_7_1167.mat
1889/2400: extracting patient_7_2308.mat
1890/2400: extracting patient_7_1601.mat
1891/2400: extracting patient_7_1615.mat
1892/2400: extracting patient_7_1173.mat
1893/2400: extracting patient_7_1198.mat
1894/2400: extracting patient_7_203.mat
1895/2400: extracting patient_7_565.mat
1896/2400: extracting patient_7_1826.mat
1897/2400: extracting patient_7_1832.mat
1898/2400: extracting patient_7_571.mat
1899/2400: extracting patient_7_217.mat
1900/2400: extracting patient_7_559.mat
1901/2400: extracting patient_7_558.mat
1902/2400: extracting patient_7_570.mat
1903/2400: extracting patient_7_1833.mat
1904/2400: extracting patient_7_216.mat
1905/2400: extracting patient_7_202.mat
1906/2400: extracting patient_7_1199.mat
1907/2400: extracting patient_7_1827.mat
1908/2400: extracting patient_7_564.mat
1909/2400: extracting patient_7_1614.mat
1910/2400: extracting patient_7_1172.mat
1911/2400: extracting patient_7_1166.mat
1912/2400: extracting pati

2104/2400: extracting patient_7_407.mat
2105/2400: extracting patient_7_1944.mat
2106/2400: extracting patient_7_1777.mat
2107/2400: extracting patient_7_1011.mat
2108/2400: extracting patient_7_1005.mat
2109/2400: extracting patient_7_1763.mat
2110/2400: extracting patient_7_2242.mat
2111/2400: extracting patient_7_1993.mat
2112/2400: extracting patient_7_1039.mat
2113/2400: extracting patient_7_1987.mat
2114/2400: extracting patient_7_2256.mat
2115/2400: extracting patient_7_1767.mat
2116/2400: extracting patient_7_1001.mat
2117/2400: extracting patient_7_1015.mat
2118/2400: extracting patient_7_1773.mat
2119/2400: extracting patient_7_2252.mat
2120/2400: extracting patient_7_1983.mat
2121/2400: extracting patient_7_1029.mat
2122/2400: extracting patient_7_1997.mat
2123/2400: extracting patient_7_2246.mat
2124/2400: extracting patient_7_359.mat
2125/2400: extracting patient_7_1968.mat
2126/2400: extracting patient_7_1798.mat
2127/2400: extracting patient_7_1940.mat
2128/2400: extract

2320/2400: extracting patient_7_1638.mat
2321/2400: extracting patient_7_2331.mat
2322/2400: extracting patient_7_2325.mat
2323/2400: extracting patient_7_548.mat
2324/2400: extracting patient_7_560.mat
2325/2400: extracting patient_7_1823.mat
2326/2400: extracting patient_7_206.mat
2327/2400: extracting patient_7_212.mat
2328/2400: extracting patient_7_1189.mat
2329/2400: extracting patient_7_1837.mat
2330/2400: extracting patient_7_574.mat
2331/2400: extracting patient_7_1188.mat
2332/2400: extracting patient_7_213.mat
2333/2400: extracting patient_7_575.mat
2334/2400: extracting patient_7_1836.mat
2335/2400: extracting patient_7_1822.mat
2336/2400: extracting patient_7_561.mat
2337/2400: extracting patient_7_207.mat
2338/2400: extracting patient_7_549.mat
2339/2400: extracting patient_7_2324.mat
2340/2400: extracting patient_7_2330.mat
2341/2400: extracting patient_7_1639.mat
2342/2400: extracting patient_7_1177.mat
2343/2400: extracting patient_7_2318.mat
2344/2400: extracting pati

In [99]:
print(data.shape)
print(labels.shape)

train_data, train_labels, val_data, val_labels = dataset.train_val_split(data, labels, .7)

print(train_data.shape)
print(train_labels.shape)
print(val_data.shape)
print(val_labels.shape)


(2707, 72)
(2707,)
(1894, 72)
(1894,)
(813, 72)
(813,)


In [107]:
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import matplotlib
import matplotlib.pyplot as plt

#### SVM
clf = SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', gamma=.00000000000000002, kernel='rbf',
    max_iter=100000, probability=True)

clf.fit(train_data, train_labels)
y_pred = clf.predict(val_data)

ab_trn = clf
from sklearn import metrics
print("training")
print(metrics.classification_report(train_labels,ab_trn.predict(train_data)))
print(metrics.roc_auc_score(train_labels, ab_trn.predict(train_data)))
print("\nvalidation")
print(metrics.classification_report(val_labels,y_pred))
print(metrics.roc_auc_score(val_labels,y_pred))
print("\n")



training
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      1677
        1.0       0.98      0.97      0.97       217

avg / total       0.99      0.99      0.99      1894

0.9826783618981668

validation
             precision    recall  f1-score   support

        0.0       0.98      0.96      0.97       723
        1.0       0.74      0.87      0.80        90

avg / total       0.96      0.95      0.95       813

0.9146611341632089




In [108]:
path = "data/patient_"+str(patient_number)+"/test/"
files = listdir(path)
print(str(len(files))+" number of points")

import os
import csv
csv_path = 'patient_roc' + str(patient_number)+'.csv'
f = open(csv_path, "w")
f.truncate()
f.close()
with open(csv_path, 'a') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i, file in enumerate(files):
        if file == ".DS_Store":
            continue
        print(str(i+1)+"/"+str(len(files))+": extracting "+file)
        file_data = spio.loadmat(path+file)["data"]
        file_data[np.isnan(file_data)] = 0
        vec = fe.extract_feature2(file_data)

        filename = file[:-4].replace("test_", "")
        score = ab_trn.predict_proba(vec.reshape(-1, 1).T)[0,1]
        pred = [filename, score]
        print(pred)

        spamwriter.writerow(pred)


4524 number of points
1/4524: extracting patient_7_test_3865.mat
['patient_7_3865', 0.06118576089154192]
2/4524: extracting patient_7_test_783.mat
['patient_7_783', 0.4528126252673621]
3/4524: extracting patient_7_test_1914.mat
['patient_7_1914', 0.3393070653196146]
4/4524: extracting patient_7_test_1900.mat
['patient_7_1900', 0.049188782346616014]
5/4524: extracting patient_7_test_797.mat
['patient_7_797', 0.2970436988220195]
6/4524: extracting patient_7_test_3871.mat
['patient_7_3871', 0.05114913175103292]
7/4524: extracting patient_7_test_967.mat
['patient_7_967', 0.7508177434800326]
8/4524: extracting patient_7_test_3681.mat
['patient_7_3681', 0.05566783548684659]
9/4524: extracting patient_7_test_3859.mat
['patient_7_3859', 0.08121416149196117]
10/4524: extracting patient_7_test_1096.mat
['patient_7_1096', 0.03331307575202341]
11/4524: extracting patient_7_test_1928.mat
['patient_7_1928', 0.26748228011530634]
12/4524: extracting patient_7_test_1082.mat
['patient_7_1082', 0.4127971

['patient_7_1860', 0.2097561162193727]
111/4524: extracting patient_7_test_1874.mat
['patient_7_1874', 0.4127971597086223]
112/4524: extracting patient_7_test_185.mat
['patient_7_185', 0.06324051948840123]
113/4524: extracting patient_7_test_3905.mat
['patient_7_3905', 0.07684265394085904]
114/4524: extracting patient_7_test_813.mat
['patient_7_813', 0.04656380708764579]
115/4524: extracting patient_7_test_1684.mat
['patient_7_1684', 0.048695271573876396]
116/4524: extracting patient_7_test_87.mat
['patient_7_87', 0.9120154781713204]
117/4524: extracting patient_7_test_3093.mat
['patient_7_3093', 0.02672164881043344]
118/4524: extracting patient_7_test_3087.mat
['patient_7_3087', 0.5]
119/4524: extracting patient_7_test_2399.mat
['patient_7_2399', 0.03366255170967639]
120/4524: extracting patient_7_test_1690.mat
['patient_7_1690', 0.049572165008374906]
121/4524: extracting patient_7_test_93.mat
['patient_7_93', 0.33583294995340535]
122/4524: extracting patient_7_test_1848.mat
['patient

['patient_7_2993', 0.04923873413133666]
230/4524: extracting patient_7_test_1242.mat
['patient_7_1242', 0.93628289654625]
231/4524: extracting patient_7_test_1524.mat
['patient_7_1524', 0.0665632848582862]
232/4524: extracting patient_7_test_3333.mat
['patient_7_3333', 0.006580365513090602]
233/4524: extracting patient_7_test_4448.mat
['patient_7_4448', 0.018867562602336158]
234/4524: extracting patient_7_test_3327.mat
['patient_7_3327', 0.06081925249191142]
235/4524: extracting patient_7_test_2039.mat
['patient_7_2039', 0.07601832063735749]
236/4524: extracting patient_7_test_219.mat
['patient_7_219', 0.1046813641569161]
237/4524: extracting patient_7_test_1530.mat
['patient_7_1530', 0.05723852887247949]
238/4524: extracting patient_7_test_1256.mat
['patient_7_1256', 0.3474984078065139]
239/4524: extracting patient_7_test_3441.mat
['patient_7_3441', 0.05376861449661921]
240/4524: extracting patient_7_test_2987.mat
['patient_7_2987', 0.6393259974295044]
241/4524: extracting patient_7_t

['patient_7_3480', 0.004227827732871893]
343/4524: extracting patient_7_test_4489.mat
['patient_7_4489', 0.0981363660637142]
344/4524: extracting patient_7_test_233.mat
['patient_7_233', 0.4127971597086223]
345/4524: extracting patient_7_test_2013.mat
['patient_7_2013', 0.21755382746374888]
346/4524: extracting patient_7_test_4462.mat
['patient_7_4462', 0.009063297201874288]
347/4524: extracting patient_7_test_4304.mat
['patient_7_4304', 0.022076096365956897]
348/4524: extracting patient_7_test_2775.mat
['patient_7_2775', 0.00043738759754572527]
349/4524: extracting patient_7_test_555.mat
['patient_7_555', 0.4129148795411476]
350/4524: extracting patient_7_test_1268.mat
['patient_7_1268', 0.08956626249471439]
351/4524: extracting patient_7_test_4310.mat
['patient_7_4310', 0.00018257273112533073]
352/4524: extracting patient_7_test_2761.mat
['patient_7_2761', 0.3918335479106749]
353/4524: extracting patient_7_test_541.mat
['patient_7_541', 0.050986990691592446]
354/4524: extracting pati

['patient_7_1651', 0.02921915836972317]
459/4524: extracting patient_7_test_1889.mat
['patient_7_1889', 0.6084467019970305]
460/4524: extracting patient_7_test_2370.mat
['patient_7_2370', 0.030637877606262957]
461/4524: extracting patient_7_test_150.mat
['patient_7_150', 0.3381069917251801]
462/4524: extracting patient_7_test_1679.mat
['patient_7_1679', 0.05646357260280689]
463/4524: extracting patient_7_test_636.mat
['patient_7_636', 0.045114753869406346]
464/4524: extracting patient_7_test_2416.mat
['patient_7_2416', 0.0469836877991379]
465/4524: extracting patient_7_test_3708.mat
['patient_7_3708', 0.04227476795336615]
466/4524: extracting patient_7_test_4067.mat
['patient_7_4067', 0.5100988388068789]
467/4524: extracting patient_7_test_622.mat
['patient_7_622', 0.05043585436640984]
468/4524: extracting patient_7_test_2402.mat
['patient_7_2402', 0.049211881674118066]
469/4524: extracting patient_7_test_4073.mat
['patient_7_4073', 0.06661784757833385]
470/4524: extracting patient_7_t

...

['patient_7_1516', 0.06965220483871794]
4008/4524: extracting patient_7_test_1270.mat
['patient_7_1270', 0.4127972853456336]
4009/4524: extracting patient_7_test_3467.mat
['patient_7_3467', 0.026397623311498943]
4010/4524: extracting patient_7_test_4308.mat
['patient_7_4308', 0.001591734297454896]
4011/4524: extracting patient_7_test_559.mat
['patient_7_559', 0.004406039077356899]
4012/4524: extracting patient_7_test_2779.mat
['patient_7_2779', 0.10369849475536576]
4013/4524: extracting patient_7_test_1258.mat
['patient_7_1258', 0.020083840434098745]
4014/4524: extracting patient_7_test_4320.mat
['patient_7_4320', 0.034335526278363414]
4015/4524: extracting patient_7_test_2989.mat
['patient_7_2989', 0.06182351287713745]
4016/4524: extracting patient_7_test_571.mat
['patient_7_571', 0.08787552863773777]
4017/4524: extracting patient_7_test_2751.mat
['patient_7_2751', 0.20377096071518605]
4018/4524: extracting patient_7_test_2037.mat
['patient_7_2037', 0.20619371787568389]
4019/4524: ext

['patient_7_2397', 0.4127971597086223]
4129/4524: extracting patient_7_test_3089.mat
['patient_7_3089', 0.02076072684929064]
4130/4524: extracting patient_7_test_1846.mat
['patient_7_1846', 0.047782745946851786]
4131/4524: extracting patient_7_test_1852.mat
['patient_7_1852', 0.4127971597086223]
4132/4524: extracting patient_7_test_89.mat
['patient_7_89', 0.06203677560632317]
4133/4524: extracting patient_7_test_2383.mat
['patient_7_2383', 0.08051793510015093]
4134/4524: extracting patient_7_test_4094.mat
['patient_7_4094', 0.0784171364179632]
4135/4524: extracting patient_7_test_3923.mat
['patient_7_3923', 0.018405768288350808]
4136/4524: extracting patient_7_test_835.mat
['patient_7_835', 0.048476057523215085]
4137/4524: extracting patient_7_test_821.mat
['patient_7_821', 0.048869640100471186]
4138/4524: extracting patient_7_test_2156.mat
['patient_7_2156', 0.05293010095580684]
4139/4524: extracting patient_7_test_376.mat
['patient_7_376', 0.06019813166584508]
4140/4524: extracting p

['patient_7_1717', 0.061046506934468905]
4249/4524: extracting patient_7_test_3100.mat
['patient_7_3100', 0.41102391082298123]
4250/4524: extracting patient_7_test_2578.mat
['patient_7_2578', 0.02508312107312525]
4251/4524: extracting patient_7_test_758.mat
['patient_7_758', 0.047985015003614996]
4252/4524: extracting patient_7_test_4109.mat
['patient_7_4109', 0.00040049664110363786]
4253/4524: extracting patient_7_test_980.mat
['patient_7_980', 0.0169826778624451]
4254/4524: extracting patient_7_test_3666.mat
['patient_7_3666', 0.007757134859874672]
4255/4524: extracting patient_7_test_1071.mat
['patient_7_1071', 0.09907192828186456]
4256/4524: extracting patient_7_test_1065.mat
['patient_7_1065', 0.3523717714255651]
4257/4524: extracting patient_7_test_3672.mat
['patient_7_3672', 0.05562266782336493]
4258/4524: extracting patient_7_test_994.mat
['patient_7_994', 0.04353421706074598]
4259/4524: extracting patient_7_test_3114.mat
['patient_7_3114', 0.07158308045609271]
4260/4524: extra

['patient_7_3048', 0.02421995826645639]
4369/4524: extracting patient_7_test_1887.mat
['patient_7_1887', 0.0017509755135884006]
4370/4524: extracting patient_7_test_1139.mat
['patient_7_1139', 0.07392714444130563]
4371/4524: extracting patient_7_test_4041.mat
['patient_7_4041', 6.870469395272845e-07]
4372/4524: extracting patient_7_test_610.mat
['patient_7_610', 0.06290320636040207]
4373/4524: extracting patient_7_test_2430.mat
['patient_7_2430', 0.06667667166549564]
4374/4524: extracting patient_7_test_4055.mat
['patient_7_4055', 0.04435017578382856]
4375/4524: extracting patient_7_test_604.mat
['patient_7_604', 0.9999999986320959]
4376/4524: extracting patient_7_test_2424.mat
['patient_7_2424', 0.04239158678923969]
4377/4524: extracting patient_7_test_1893.mat
['patient_7_1893', 0.059237288364370526]
4378/4524: extracting patient_7_test_48.mat
['patient_7_48', 0.05943729929888744]
4379/4524: extracting patient_7_test_2342.mat
['patient_7_2342', 0.07843719734098817]
4380/4524: extract

['patient_7_2551', 0.08606808594986624]
4494/4524: extracting patient_7_test_2237.mat
['patient_7_2237', 0.052003769590076854]
4495/4524: extracting patient_7_test_3129.mat
['patient_7_3129', 0.0794437656085481]
4496/4524: extracting patient_7_test_2223.mat
['patient_7_2223', 0.06118095163563088]
4497/4524: extracting patient_7_test_4134.mat
['patient_7_4134', 0.41339168511677876]
4498/4524: extracting patient_7_test_765.mat
['patient_7_765', 0.03211838077172415]
4499/4524: extracting patient_7_test_3883.mat
['patient_7_3883', 0.12857630350162733]
4500/4524: extracting patient_7_test_2545.mat
['patient_7_2545', 0.07601940846358701]
4501/4524: extracting patient_7_test_995.mat
['patient_7_995', 0.5270241403718571]
4502/4524: extracting patient_7_test_3673.mat
['patient_7_3673', 0.15494069757700835]
4503/4524: extracting patient_7_test_1064.mat
['patient_7_1064', 0.07281117490183565]
4504/4524: extracting patient_7_test_1702.mat
['patient_7_1702', 0.047848022547273085]
4505/4524: extract

In [117]:
###concatenate all files
import pandas as pd
files = ["patient_roc1.csv","patient_roc2.csv","patient_roc3.csv",
         "patient_roc4.csv","patient_roc5.csv","patient_roc6.csv","patient_roc7.csv"]
merged = []
for f in files:
    read = pd.read_csv(f)
    merged.append(read)

result = pd.concat(merged, ignore_index=True, )
result.to_csv('patient.csv')
    

In [ ]:
#### bagging
#rf_trn = RandomForestClassifier(n_estimators = m, criterion = "gini")           
#rf_trn = rf_trn.fit(train_data, train_labels)
#y_pred = rf_trn.predict(val_data)

###Adaboost
#d = 5
#wc = 50
'''
classifier_vals = [5, 10, 25, 50, 100, 150, 200]
acc_trn = []
for wc in classifier_vals:
#tree_adaboost = DecisionTreeClassifier(max_depth=d) #, random_state = 0
    ab_trn = AdaBoostClassifier(base_estimator = clf, n_estimators = wc)           
    ab_trn = ab_trn.fit(train_data, train_labels)
    y_pred = ab_trn.predict(val_data)
    acc_trn.append(ab_trn.score(train_data, train_labels))

    
plt.figure()
plt.plot(classifier_vals, acc_trn, color = 'red')
plt.title('Adaboost: acc vs weak classifiers')
   # plt.xticks(np.arange(min(classifier_vals), max(classifier_vals)+1, 10))
plt.xlabel('weak classifiers')
plt.ylabel('accuracy')

from sklearn import metrics
print("training")
print(metrics.classification_report(ab_trn.predict(train_data), train_labels))
print(metrics.roc_auc_score(ab_trn.predict(train_data), train_labels))
print("\nvalidation")
print(metrics.classification_report(y_pred, val_labels))
print(metrics.roc_auc_score(y_pred, val_labels))
print("\n")
'''